In [ ]:
from urllib import request
from bs4 import BeautifulSoup
from random import randint
import time
import json
import glob
import face_recognition
import os
from PIL import Image, ImageOps
import numpy as np
from tqdm.notebook import tqdm
from selenium import webdriver
import pandas as pd
import shutil

In [ ]:
with open('config/actress_actors.json', 'r') as fp:
    list_act = json.load(fp)

In [ ]:
len(list_act)

In [ ]:
driver = webdriver.Chrome(executable_path=r"C:\Users\zayse\Desktop\actor\config\chromedriver.exe")

In [ ]:
def downloadimages(name, count, parent_dir):
    str_face = f'face%20{name}'.replace(' ', '%20')
    dir_ = f'{parent_dir}/{name}/'
    count_of_photo = 0

    if os.path.exists(dir_):
        files = glob.glob(f'{parent_dir}/{name}/*')
        print(f'Path almost exist {dir_}')
        if len(files) > 0:
            print(f'Download {len(files)} photo {name} soon')
            print()
            return
    else:
        os.makedirs(dir_)

    driver.get(f'https://yandex.ru/images/search?text={str_face}')

    for i in range(1, count // 10):
        driver.execute_script(f'window.scrollTo(0, {i * 800})')
        time.sleep(randint(2, 5))

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    for text in soup.findAll(attrs={'class': 'serp-item__thumb justifier__thumb'}):
        if count_of_photo == count:
            break
        url = "http:" + text.get('src')
        request.urlretrieve(url, dir_ + str(count_of_photo) + ".jpg")
        count_of_photo += 1

    print(f'Download {count_of_photo} images of {name}')
    print()
    
    time.sleep(randint(5, 10))

In [ ]:
for name in tqdm(list_act):
    #downloadimages(name, 200, 'dataset')
    downloadimages(name, 1, 'examples')

In [ ]:
for name in tqdm(list_act):
    if os.path.exists(f'dataset/{name}'):
        files = glob.glob(f'dataset/{name}/*')
        path = f'dataset/{name}/'
        count = 0
        for file in files:
                if file != '.DS_Store':
                    img = Image.open(file)

                    width, height = img.size
                    if width == height == 200:
                          continue
                    face = np.array(img)

                    coordinates = face_recognition.face_locations(face)

                    if len(coordinates) == 1:

                        top, right, bottom, left = coordinates[0]

                        crop = (left - left // 2, top - top // 2, right + (width - right) // 2, bottom + (height - bottom) // 2)

                        img_crop = img.crop(crop)

                        img_crop = ImageOps.fit(img_crop, (200, 200), method = 0, bleed = 0.0, centering =(0.5, 0.5))

                        img_crop.save(f'{path}img{count}.jpg')
                        os.remove(file)
                        count += 1
                    else:
                        print(f'Pic {file} is error')
                        os.remove(file)
                else:
                    print(f'Pic {file} is error')
                    os.remove(file)

        print(f'{name} is done. {count} pic')
        print()

In [ ]:
count_of_photo = pd.DataFrame(columns=['name', 'count'])

for name in list_act:
    if os.path.exists(f'dataset/{name}'):
        files = len(glob.glob(f'dataset/{name}/*'))
        count_of_photo = count_of_photo.append({'name' : name,
                                                'count' : files},
                                               ignore_index = True)

In [ ]:
count_of_photo.sort_values(by=['count'])

In [ ]:
del_photo = count_of_photo.loc[count_of_photo['count'] < 80]
del_photo

In [ ]:
for i in del_photo['name']:
    path = f'dataset/{i}/'
    shutil.rmtree(path)

In [ ]:
files = glob.glob(f'img_test/*')
for file in files:

    img = Image.open(file)
    width, height = img.size

    if width == height == 200:
        continue

    face = np.array(img)

    coordinates = face_recognition.face_locations(face)
    if len(coordinates) == 1:

        top, right, bottom, left = coordinates[0]

        crop = (left - left // 2, top - top // 2, right + (width - right) // 2, bottom + (height - bottom) // 2)

        img_crop = img.crop(crop)

        img_crop = ImageOps.fit(img_crop, (200, 200), method = 0, bleed = 0.0, centering =(0.5, 0.5))
        img_crop.save(file)
    else:
        os.remove(file)